<a href="https://colab.research.google.com/github/JuliBaCSE/DeepLearning/blob/CNN/convolutional_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network Dog vs Cat

### Importing the libraries



In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.4.1'

## Part 1 - Data Preprocessing
As input we have 4000 dog and 4000 cat images for training and for testing we have 1000 of each

### Preprocessing the Training set

First we are going to apply transformations on our training set to avoid overfitting (high accuracy on training set and low accuracy on test set). 

! do not on test set !

For Transformations we are doing some simple geometric transformations (zoom, rotation, flip,...). This will give us new images (augmented) and therefore the CNN does not overlearn.

to do this we use from keras the imagedatagenerator class. this allow us real time data augmentation.

in our example we use the shear, zoom, flip

with rescale we get all pixel values between 0 an 1 

In the second part we import the images from the training set and resize (to target size) to reduce computations. Batch size tells how many images are in one batch. Since the outcome is either cat or dog (binary) we use class mode binary

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

### Preprocessing the Test set

since we want to keep our test images as they are to avoid information leackage we dont apply the transformations here. !!But we still have to rescale the pixel values to have later the same scaling in the CNN

finally we import our testset images to our notbook

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

## Part 2 - Building the CNN

### Initialising the CNN
as for the ann we again use the sequential class to creat an ann with sequence of layers


In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution
Now we use instead of a dense layer (compare ann) we use the Conv2D class. 

Here we specify the number of feature detectures with filters = ... 

the size of those feature detectures in kernel_size (a feature detecture is always quadratic here squared array (size x size)).

the rectifier activation function 



and the input shape for the very first layer

here we specify the size (since we reshaped to 64 x 64 pixel we plug this in)

and since use RGB pictures we use 3 in the end 

!! for black and white we would use 1

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

in pooling we use the maxpooling which is often the better choice as seen in the paper of Dominik Scherer Uni Bonn

Therefore we add a obj from the layer class MaxPool2D:

pool_size = frame were we take out the max (here 2 x 2)  

strides = is number of pixels the frame is shifted to the right/bottom. (often stride 2 is used)


In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

now the dont need the input shape paramter since it isnt the first layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

finally we want to flatten into a 1D array to use it as input in our ann

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

for the full connection we add a new dense layer that gets connected to our flattened layer. 



In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

since we have either dog or cat we only need one unit as output

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set



In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

## Part 4 - Making a single prediction

since we only imported something specific from the imagre lib we now need to import image again for the tools that we need now.

first we will load the image from our folder that we want to use for the test and resize it to the size we used in our model for training/test

since our predict methods need a 2D array as input we have to convert our test image from PIL-fromat to an array

since we trained with batches (each contains 32 pictures as we defined) we have a extra dimension. therefore we have to add an extra dimension that corresponds to the batch (therefore we use numpy). axis = 0 means that we add that extra dimension as the first dimension. 

Now have the right fromat to input it into the predict method

the result will contain 0 and 1 

training_set.class_indices will give us the indices for dog and cat (which is 1 and which is 0)

we first access the batch with [0] and then access the first element inside this batch [0] which contains the prediction

Since dog is 1 and cat is 0 we can use an if statement that tell us if it is a dog or cat



In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [ ]:
print(prediction)